# Lab Session Notebook - Returns

## From Prices to Returns

In this lab we'll work the very basics of Returns - computing returns, and compounding a sequence of returns.

Let's start with a set of prices for a stock "A", in a python list:

In [1]:
prices_a = [8.70, 8.91, 8.71]

Recall that the return from time $t$ to time ${t+1} is given by:

$$ R_{t,t+1} = \frac{P_{t+1}-P_{t}}{P_{t}} $$

or alternately

$$ R_{t,t+1} = \frac{P_{t+1}}{P_{t}} - 1 $$

If you come from R or another language that supports vectors, you might expect something like this to work:

```python
returns_a = prices_a[:-1]/prices_a[1:] - 1
```

However, since Python lists do not operate as vectors, that will not work, generating an error about "/" not working for lists.


In [2]:
# WILL NOT WORK - THIS WILL GENERATE AN ERROR!
prices_a[1:]/prices_a[:-1] -1

TypeError: unsupported operand type(s) for /: 'list' and 'list'

Instead, we can convert them to a `numpy` array. Numpy arrays _do_ behave like vectors, so this works:

In [10]:
import numpy as np

prices_a = np.array([8.70, 8.91, 8.71])
prices_a
print(prices_a[-1])
print(prices_a[-2])
print(prices_a[-3])

8.71
8.91
8.7


In [4]:
prices_a[1:]/prices_a[:-1] - 1

array([ 0.02413793, -0.02244669])

Now, let's add a few more days of prices and introduce a second stock. Let's call these two stocks "BLUE" and "ORANGE". Instead of using raw numpy arrays, we are going to use the far more powerful Pandas DataFrame, which wraps the functionality of numpy into a very convenient and easy to use data structure called a DataFrame. Note how the DtaFrame has two nicely indexed columns as well as a row index that by default runs from 0 to 4.

In [ ]:
import pandas as pd

prices = pd.DataFrame({"BLUE": [8.70, 8.91, 8.71, 8.43, 8.73],
                       "ORANGE": [10.66, 11.08, 10.71, 11.59, 12.11]})

In [ ]:
prices

**WARNING**

However, because Pandas DataFrames will align the row index (in this case: 0, 1, 2, 3, 4) the exact same code fragment will not work as you might expect.  (see the section on row alignment in the "Crash Course" videos if this is unclear to you)

In [ ]:
prices.iloc[1:]

In [ ]:
prices.iloc[:-1]

In [ ]:
prices.iloc[1:]/prices.iloc[:-1] - 1

We can fix this in one of several ways. First, we can extract the values of the DataFrame column which returns a numpy array, so that the DataFrame does not try and align the rows.

In [ ]:
prices.iloc[1:].values/prices.iloc[:-1] - 1

You could have also used the values in the denominator:

In [ ]:
prices.iloc[1:]/prices.iloc[:-1].values - 1

However, there are a couple of ways to do this without extracting the values, and these are probably a bit cleaner and more readable. The first option is to use the `.shift()` method on the array, which realigns the indices.

In [ ]:
prices

Since we want to get the row at index 0 (8.84 and 10.66) to line up with the row at index 1 (8.54 and 10.30) so we can divide the 2nd row (at index 1) by the first row (at index 0) we want to shift the rows in the denominator by 1 ... which we do with `.shift(1)`

In [ ]:
prices.shift(1)

So, now we can obtain the returns on each day as follows:

In [ ]:
returns = prices/prices.shift(1) - 1
returns

Note how we cannot compute returns for the first day, because we dont have the closing price for the previous day. In general, we lose one data point when we go from prices to returns.

Finally, there is a built-in method in DataFrame that computes the percent change from one row to another. Since that is exactly what a return is (the percent change in price) we can just use this method to compute the return.


In [ ]:
returns = prices.pct_change()
returns

## Reading data from a CSV file
Since typing in returns is tedious, let's read the data in from a file. Pandas provides a convenient and simple way to read in a CSV file of the returns.

In [ ]:
prices = pd.read_csv('../data/sample_prices.csv')
prices

In [ ]:
returns = prices.pct_change()
returns

In [ ]:
returns.mean()

In [ ]:
returns.std()

In [ ]:
returns.plot.bar()

In [ ]:
prices.plot()

## Compounding Returns

Now that we have a series of 12 monthly returns, we can produce the compounded return by multiplying the individual period returns, as long as the returns are expressed as growth rates in what I call "1+R" format.

To compound the returns, all we need to do is add 1 to each return and then multiply them. The result is itself in "1+R" format, so we need to subtract 1.

Let's compute the compounded return of our two series. 

In [ ]:
returns + 1

In [ ]:
np.prod(returns+1)

In [ ]:
(returns+1).prod()

In [ ]:
(returns+1).prod()-1

In [ ]:
(((returns+1).prod()-1)*100).round(2)

## Annualizing Returns

To annualize a return for a period, you compound the return for as many times as there are periods in a year. For instance, to annualize a monthly return you compund that return 12 times. The formula to annualize a monthly return $R_m$ is:

$$ (1+R_m)^{12} - 1$$

To annualize a quarterly return $R_q$ you would get:

$$ (1+R_q)^{4} - 1$$

And finally, to annualize a daily return $R_d$ you would get:

$$ (1+R_d)^{252} - 1$$

For example, to annualize a 1% monthly, and 4% quarterly and a 0.01% daily return you would do:

In [ ]:
rm = 0.01
(1+rm)**12 - 1

In [ ]:
rq = 0.04
(1+rq)**4 - 1